---
---
# Import to DataBase
---
---

In [76]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess
import __utils__

#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import re
from math import ceil

# import plotly.io as pio
# pio.renderers.default = "browser"

# Initialize DB table

---

Code not yet turned to a function.

---
---

# Import CSV files to Table

## Set path to directory where files are located

---

`path='/mnt/c/Users/16162/Documnets/Data/mydata/`

In [ ]:
path='/mnt/c/Users/16162/Documents/Data/Arsenic/'
filelist=[file for file in os.listdir(path) if '.csv' in file]
mylist=[file.split('-Frame-')[0] for file in filelist] #Check files with/out frames
for file in mylist:
    print(file)

## Extract info from CSV then send to Table

___

New functions should be written to manage this process.

In [ ]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/users/16162/Desktop/dopamine.db','arsenic')
filelist=[file for file in os.listdir(path) if '.csv' in file]

mylist=[]

for file in filelist:
    df=pd.read_csv(path+file, header=None)
    x=df.iloc[:,0]
    x=[int(i) for i in x]
    x=np.array(x)
    xb=x.tobytes()
    
    y=df.iloc[:,1]
    y=np.array(y)
    yb=y.tobytes()

    splits=file.split('_')
    specnum=int(splits[0])
    frame = int(file.split("-")[2].split(".")[0])
    as3='0'
    as5='0'


    if 'blank' in file.lower():
        pass
    elif len(file.split(' ')) == 2:
        concen=splits[1]
        if 'as3' in splits[2].lower():
            as3=concen
        elif 'as5' in splits[2].lower():
            as5=concen
    elif len(file.split(' ')) ==3:
        for split in splits:
            if 'as5' in split.lower():
                as5=split.replace('AS5acsf','')
            elif 'as3' in split.lower():
                si=splits.index(split) #index for the split
                as3=splits[si-1]#value in the split before 's'
    else:
        print('problem: ', file)

    print(f'filename: {file}\nframe: {frame}\nas3: {as3}\nas5: {as5}\nspecnum: {specnum}')
    print(xb)
    print(yb)
    print()
    # db.insert_row(('raman_shift', xb),('original_intensity', yb),
    #               ('As3',as3), ('As5',as5), ('filename',file),
    #               ('frame',frame),('specnum',specnum))


# Edit values in database

___

Add notes

In [57]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
db.columns('general_loc')[1]
# db.update('general_loc','cerebral cortex', 'cortex')


Values in general_loc:
	blank
	center
	slice2blank
	slice2wellplatediffplateposition
	slice2wellplate
	slice2
	slice3coverslip
	subniagraish
	cortex
	midbrain
	substantia nigra
	vta
	brainstem
	tissue2
	tissue3
	532nm
	ventrical
	low


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,539,INTEGER,0,None,1
1,raman_shift,12,BLOB,0,None,0
2,original_intensity,446,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,222,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0


# Also add a test function to make sure the import worked. Then cool it with these headers